# Feed Forward Neural Network with Back Propagation

The objective of this notebook is to implement smartly and scalably a fully connected neural network that will be trained by means of a gradient descent back propagation algorithm. Afterwards I will test some simple networks on small data sets and try to run a regression analysis predicing milage per galon (mpg) based on the mtcars data set.

* Get the mtcars data, bulid features and also divide into train and test. 
* Make some simple linearely separable and XOR data test cases. 
* Create classes and methods for marticular modules - linear and activation of the neural network
* Create a neural network class which is build out of modules and has a training method. 
* Run the simple test and train the network on mtcars and see if we are getting any reasonable results.


### Data preparation

Get mtcars data and do feature preparation for a regression problem. The NN network will try to learn mpg

In [951]:
import numpy as np
import pandas as pd
import csv
import itertools
from matplotlib import pyplot as plt
from math import exp

def load_data(path):
    '''
    takes path and returns a pnadas data frame object with the data from file under path
    '''
    data = []
    with open(path) as f:
        for row in csv.DictReader(f, delimiter='\t'):
            data.append(row)
    return pd.DataFrame(data)


def one_hot(x):
    '''
    @param x : pandas series object
    returns a data frame with the original series and the one hot encoded fields
    '''
    
    new_frame = pd.DataFrame(x)
    for val in x.unique():
        _name = str(int(val))
        new_frame[x.name + _name] = new_frame.apply(lambda col: 1 if col[x.name] == val else 0, axis=1)
    return new_frame.iloc[:,1:]
        
    
def standardize(x):
    '''
    @param x : string holding the name of the field to be standarized
    '''
    z = x.astype('float')
    result = np.array((z - np.mean(z))/np.std(z))

    return pd.DataFrame(result, columns=[x.name])


def make_features(feature_list):
    '''
    @param feature_list: a list of tuples, first entry is a pandas series and the next one is a string with 
    onehot or standardize. 
    '''
    new_features = []
    for f,ftype in feature_list:
        if ftype == 'onehot':
            new_features.append(one_hot(f))
        elif ftype == 'standardize':
            new_features.append(standardize(f))
        else:
            new_features.append(f.astype('float'))
        
    return pd.concat(new_features, axis=1)



auto_data = load_data('../code_and_data_for_hw05/auto-mpg-regression.tsv')


features = [
            (auto_data.cylinders, 'onehot'),
            (auto_data.displacement, 'standardize'),
            (auto_data.horsepower, 'standardize'),
            (auto_data.weight, 'standardize'),
            (auto_data.acceleration, 'standardize'),
            (auto_data.origin, 'onehot'),
            (auto_data.mpg, 'standardize')
            ]


# Keep this for future reference:
mean_mpg, sigma_mpg = auto_data['mpg'].astype('float').mean(), auto_data['mpg'].astype('float').std() 


auto_data_ = make_features(features)



# Transpose  data and divide into training and testing sets:
X = (auto_data_.T).iloc[:-1, :]
Y = (auto_data_.T).iloc[[-1], :]

# Divide at random into test and train. The test data set will be approzimately 20% of the entire data set
test_ind = np.random.choice(range(X.shape[1]), int(0.2 * X.shape[1]), replace = False)
train_ind = np.array(list(set(range(X.shape[1]))- set(test_ind)))

X_test, Y_test = np.array(X.iloc[:, test_ind]), np.array(Y.iloc[:, test_ind])
X_train, Y_train = np.array(X.iloc[:, train_ind]), np.array(Y.iloc[:, train_ind])




Build versy simple data set that is one dimensional and linearly separable. In fact values above 2 are classified as 1 and values below 2 as zero.  We want to see if the network can classify correctly. Thise is actually an implementation test case. If all is good, the network should be able to handle this classification task.

In [981]:
X_simple_1D = np.array([[1.4, -1.5, -0.9, 2.7, 4.1, 5.0 ], [1,1,1,1,1,1]])
Y_simple_1D = np.array([[0,0,0,1,1,1]])

Same as above, but this time we go for two dimensions. Poinst for which $x<0 \land y>0$ will be classified as 0 and points for which $x>0 \land y<0$ as 1. This is another implementation test indeed. 

In [980]:
X_simple_2D = np.array([[0.4, -1.5, -0.9, 0.9, 1.2, -1.2, 3.1], 
                        [0.5, -1, -0.4, 1.4, 0.7, -0.5, 1.0] , 
                        [1, 1, 1, 1, 1, 1, 1]])
Y_simple_2D = np.array([[1,0,0,1,1,0,1]])

And the last imlementaton test is the XOR test. Here we check if the network can learn the XOR gate

In [982]:
X_xor = np.array([[0,0, 1, 1], [0, 1, 0, 1], [1, 1, 1, 1]])
Y_xor = np.array([[0, 1, 1, 0]])


### Neural Network Implementation

Implementation of the linear module with forward and backward methods

In [1022]:
class Linear(object):
    def __init__(self, m,n):
        '''
            This is the linear part that takes in m inputs, multiplies by weights and produces n outputs
            m - number of inputs
            n - number of outputs
        '''
        self.m = m
        self.n = n
        self.W = np.random.normal(loc=0.0, scale=3 * m ** (-.5), size=(n,m)).reshape((self.n, self.m))
        self.W[[n-1],:] = np.zeros((1,self.m))
        
    def forward(self, X):
        '''
            X are the inputs. The forward method will produce and save activations and also return them
        '''
        assert X.shape[0] == self.m
        self.A = X  # if self is layer l, then self.A is l-1 layer, the inputs to this module
        self.Z  = np.matmul(self.W, X)
        return self.Z
    
    def backward(self, dLdZ):
        '''
            dLdZ is passed from the activation layer. We can compyte dLdA = dLdZ * dZdA, but dZdA = W
            and pass it to the previous activation layer. 
            knowing dLdZ, we compute dLdW = dLdZ * dZdW = dLdZ * A(l-1)  and store it
        '''
        self.dLdA = np.matmul((self.W).T, dLdZ)
        self.dLdW = np.matmul(self.A, dLdZ.T).T
        return self.dLdA
    
    def update(self, lrate):
        '''
            just update weights 
        '''
        assert self.W.shape == self.dLdW.shape
        self.W = self.W - lrate * self.dLdW

Implementation of the ReLU, sigmoid and identity activations with forward and backward methods

In [1139]:
class ReLU(object):
    def __init__(self):
        pass
    
    def forward(self, X):
        '''
            Take an X as input and apply ReLU elementwise
        '''
        def ReLU(x):
            return x if x>=0 else 0
        self.A = X
        self.Z = np.vectorize(lambda x: ReLU(x))(self.A)
        return self.Z

    def backward(self, dLdA):
        '''
            Takes dLdA, where dA means with respect to the output of the module.  
            Returns dLdZ (Z is the input to the module). dLdZ = dLdA * dAdZ , but dAdZ is ReLU_grad
        '''
        def ReLU_grad(x):
            return 1 if x>=0 else 0
        return dLdA * np.vectorize(lambda x: ReLU_grad(x))(self.A) 


class Sigmoid(object):
    def __init__(self):
        pass
    
    def forward(self, X):
        '''
            Take an X as input and apply ReLU elementwise
        '''
        def sigmoid(x):
            return 1/(1+exp(-x))
        self.A = X
        self.Z = np.vectorize(lambda x: sigmoid(x))(self.A)
        return self.Z

    def backward(self, dLdA):
        '''
            Takes dLdA, where dA means with respect to the output of the module.  
            Returns dLdZ (Z is the input to the module). dLdZ = dLdA * dAdZ , but dAdZ is ReLU_grad
        '''
        def sigmoid(x):
            return 1/(1+exp(-x))

        def sigmoid_grad(x):
            return sigmoid(x)*(1 - sigmoid(x))

        return dLdA * np.vectorize(lambda x: sigmoid_grad(x))(self.A) 

    
class Identity(object):
    def __init__(self):
        pass
    
    def forward(self, X):
        '''
            Take an X as input and apply ReLU elementwise
        '''
        self.A = X
        self.Z = X
        return self.Z

    def backward(self, dLdA):
        '''
            Takes dLdA, where dA means with respect to the output of the module.  
            Returns dLdZ (Z is the input to the module). dLdZ = dLdA * dAdZ , but dAdZ is ReLU_grad
        '''
        return dLdA



Implementation of quadratic and negative log likelyhood loss modules, with loss and loss_grad methods

In [1144]:
class Quadratic_loss():
    def __init__(self):
        pass
        
    
    def loss(self, Ypred, Y):
        '''
            simple quadratic loss. In general this should be a scalar
        '''
        return ((Ypred  - Y)**2)#.sum(axis=1, keepdims=True)/(Ypred.shape[1])
    
    def loss_grad(self, Ypred, Y):
        '''
            quadratic loss gradient. In general this should be a scalar
        '''
        self.A = Ypred
        self.dLdA = 2*(Ypred - Y)#.sum(axis=1, keepdims=True)/(Ypred.shape[1])
        return self.dLdA

    
class NLL_loss():
    def __init__(self):
        pass
    
    def loss(self, Ypred, Y):
        '''
            simple quadratic loss. In general this should be a scalar
        '''

        return - Y*np.log(Ypred) - (1-Y)*np.log(1 - Ypred)
    
    def loss_grad(self, Ypred, Y):
        '''
            quadratic loss gradient. In general this should be a scalar
        '''
        
        self.A = Ypred
        self.dLdA = - Y*(1 / Ypred) + (1 - Y)*(1/(1 - Ypred))#.sum(axis=1, keepdims=True)/(Ypred.shape[1])
        return self.dLdA



Flinally comes the implementation of the entire network with forward pass, backward pass and a train method that uses a stochastic gradient descent to train the netowrk

In [831]:
class nn():
    def __init__(self, modules, loss_module):
        '''
            modules - list of alternating linear and activation modules
            loss_module - the final loss module of the network
        '''
        self.modules = modules
        self.loss_module = loss_module
        
    def forward(self, X):
        '''
            takes input X and makes a forward path returning the last activation, i.e. the output of the nn
        '''
        L = len(self.modules)
        for m in range(L):
            Xf = self.modules[m].forward(X)
            X = Xf
        return X
    
    def backward(self, dLdA):
        '''
            takes as input dLdA, where A is the output of the network, and performs a backward pass
        '''
        L = len(self.modules)
        delta = dLdA
        for m in range(L-1,-1,-1):
            delta_new = self.modules[m].backward(delta)
            delta = delta_new
    
    def train(self, X_train, Y_train, num_iter, lrate):
        '''
            Train the network using SGD picking one data point from X_train, Y_train
        '''
        for i in range(num_iter):
            t = np.random.randint(X_train.shape[1])

            Y_pred = self.forward(X_train[:,[t]])
            
            delta  = self.loss_module.loss_grad(Y_pred, Y_train[:,[t]])

            self.backward(delta)
            for module in self.modules:
                if 'W' in dir(module):
                    module.update(lrate)
        


### Testing the nueral network algorithm on some basic examples

In [1145]:
# My test function
def test_nn(X, Y, NN, num_iter, lrate):
    '''
        takes an MM, training set X, labels set Y, num_iter and learning rate as shows how the NN did learning
    '''
    print('------Ypred---- and--loss--BEFORE---training----\n')
    Ypred = NN.forward(X)
    print('prediction: ',  Ypred)
    loss = NN.loss_module.loss(Ypred, Y)
    print('loss: ', loss.sum())
    print('------Ypred---- and--loss--AFTER---training----\n')

    NN.train(X, Y, num_iter, lrate)
    Ypred = NN.forward(X)
    print('prediction ', Ypred)
    loss = NN.loss_module.loss(Ypred, Y)
    print('loss: ', loss.sum())

    print('---Ypred---and--Y--side--by--side-----\n ')
    print(np.vstack((Ypred, Y)).T)



Just one dimension detect if data is above or below zero

In [1146]:
# Basic simple one dimensional setting to see if some fundamentals work
Lin1 = Linear(2,1)
#Lin1.W = np.array([[0.6,-0.5]])
S1 = Sigmoid()
L = NLL_loss()
NN = nn([Lin1, S1], L)
test_nn(X_simple_1D, Y_simple_1D, NN, 5000, 0.04)

------Ypred---- and--loss--BEFORE---training----

prediction:  [[0.5 0.5 0.5 0.5 0.5 0.5]]
loss:  4.1588830833596715
------Ypred---- and--loss--AFTER---training----

prediction  [[0.16061856 0.00004506 0.00025372 0.89004036 0.99781511 0.99983617]]
loss:  0.29422844389201586
---Ypred---and--Y--side--by--side-----
 
[[0.16061856 0.        ]
 [0.00004506 0.        ]
 [0.00025372 0.        ]
 [0.89004036 1.        ]
 [0.99781511 1.        ]
 [0.99983617 1.        ]]


Linearly seprable set in two dimensions. 

In [1147]:
# Linearly separable but this time two dimensional set 
np.set_printoptions(suppress=True)
Lin1 = Linear(3,3)
'''
Interestingly, this network requires some hand picking while initializing weights in the the two linear layers
Without this hand adjustment it does not perform well. I guess that the original randomization produces too small values
'''
Lin1.W = np.array([[0.3, -2, 1.5], [-4.2, -0.9, 1.8], [1, -0.5, 0.9]])
S1 = ReLU()
Lin2 = Linear(3, 1)
Lin2.W = np.array([[1,2, -0.5]])
S2 = Sigmoid()
L = NLL_loss()
NN = nn([Lin1, S1, Lin2, S2], L)
test_nn(X_simple_2D, Y_simple_2D, NN, 1000, 0.04)

------Ypred---- and--loss--BEFORE---training----

prediction:  [[0.52373215 1.         0.99999899 0.36586441 0.39771382 0.99999995
  0.21081829]]
loss:  55.71104964235848
------Ypred---- and--loss--AFTER---training----

prediction  [[0.9912327  0.00004586 0.00510379 0.9930691  0.99922541 0.00042403
  0.99999821]]
loss:  0.0221245097686201
---Ypred---and--Y--side--by--side-----
 
[[0.9912327  1.        ]
 [0.00004586 0.        ]
 [0.00510379 0.        ]
 [0.9930691  1.        ]
 [0.99922541 1.        ]
 [0.00042403 0.        ]
 [0.99999821 1.        ]]


Testing the XOR data set

In [1148]:
'''
Caution: As before, default weight initialization does not seem to give good results
One needs to tweak that initialization and lrate to get good results
'''
Lin1 = Linear(3,8)
Lin1.W = np.random.normal(loc = 0., scale=2, size=(8,3))
S1 = ReLU()
Lin2 = Linear(8, 1)
Lin2.W = np.random.normal(loc = 0., scale=3, size=(1,8))
S2 = Sigmoid()
L = NLL_loss()
NN = nn([Lin1, S1, Lin2, S2], L)
test_nn(X_xor, Y_xor, NN, 3000, 0.005)

------Ypred---- and--loss--BEFORE---training----

prediction:  [[0.24113505 0.00033206 0.99712186 0.35810752]]
loss:  8.732330501143595
------Ypred---- and--loss--AFTER---training----

prediction  [[0.03499658 0.85405451 0.97004196 0.00444279]]
loss:  0.22825252891766884
---Ypred---and--Y--side--by--side-----
 
[[0.03499658 0.        ]
 [0.85405451 1.        ]
 [0.97004196 1.        ]
 [0.00444279 0.        ]]


#### Neural Network Regression with the mtcars data set

In [1150]:
# Network with two linear modules followed by ReLU activation and a quadratic loss module
L1 = Linear(12, 36)
R1 = ReLU()
L2 = Linear(36,24)
R2 = ReLU()
L3 = Linear(24, 1)
R4 = Identity()
Q = Quadratic_loss()
NN = nn([L1, R1, L2, R2, L3, R4], Q)


Y_pred = NN.forward(X_train)

Y_pred_original = Y_pred
Y_pred_original_test = NN.forward(X_test)


print('loss before training ', Q.loss(Y_pred, Y_train).sum())
NN.train(X_train, Y_train, 15000, 0.0025)

Y_pred = NN.forward(X_train)
print('loss after training ', Q.loss(Y_pred, Y_train).sum())
RMSE = ((Y_pred * sigma_mpg - Y_train * sigma_mpg)**2).sum()/Y_train.shape[1]
RMSE = RMSE**0.5 
print('The RMSE is {:.2f}'.format(RMSE))


loss before training  299.8733737339507
loss after training  84.12500605463816
The RMSE is 4.04


In [1133]:

pd.DataFrame(np.vstack((Y_pred_original*sigma_mpg+mean_mpg, Y_pred*sigma_mpg+mean_mpg, Y_train*sigma_mpg+mean_mpg)).T, \
             columns=['Y_pred_original', 'Y_pred_trained', 'Y_train'])


,Y_pred_original,Y_pred_trained,Y_train
0,23.445918,14.362293,14.989206
1,23.445918,16.308400,17.993040
2,23.445918,15.906414,15.990484
3,23.445918,14.610603,14.989206
4,23.445918,15.515206,13.987929
...,...,...,...
309,23.445918,24.003141,27.004542
310,23.445918,38.044107,44.026267
311,23.445918,32.535737,32.010932
312,23.445918,26.613805,28.005820


Now we let's see what will be the RSE on the test data set

In [1134]:
Y_pred = NN.forward(X_test)
RMSE = ((Y_pred*sigma_mpg-Y_test*sigma_mpg)**2).sum()/Y_test.shape[1]
RMSE = RMSE**0.5 
print('The RMSE on the test data set after training is {:.2f}'.format(RMSE))


RMSE_org = ((Y_pred_original_test*sigma_mpg-Y_test*sigma_mpg)**2).sum()/Y_test.shape[1]
RMSE_org = RMSE_org**0.5 
print('The RMSE on the test data before training was {:.2f}'.format(RMSE_org))



The RMSE on the test data set is 3.79
The RMSE on the test data before training was 8.48
